# Appendix 10.2.2: Forcing JSON with tool use

## Learning goals

* Understand using tools to force a structured response
* Utilize this "trick" to generate structured JSON

One of the more interesting ways of utilizing tool use is in forcing Claude to respond with structured content like JSON.  There are many situations in which we may want to get a standardized JSON response from Claude: extracting entities, summarizing data, analyzing sentiment, etc. 

One way of doing this is simply asking Claude to respond with JSON, but this can require additional work to actually extract the JSON from the big string we get back from Claude or to make sure the JSON follows the exact format we want.

The good news is that **whenever Claude wants to use a tool, it already responds using the perfectly structured format we told it to use when we defined the tool.**

In the previous lesson, we gave Claude a calculator tool.  When it wanted to use the tool, it responded with content like this: 

```
{
    'operand1': 1984135, 
    'operand2': 9343116, 
    'operation': 'multiply'
}
```

That looks suspiciously similar to JSON! 

If we want Claude to generate structured JSON, we can use this to our advantage.  All we have to do is define a tool that describes a particular JSON structure and then tell Claude about it.  That's it.  Claude will respond back, thinking it's "calling a tool" but really all we care about is the structured response it gives us.

***

# Conceptual overview

How is this different than what we did in the previous lesson?  Here's a diagram of the workflow from the last lesson: 

![chickens_calculator.png](./images/chickens_calculator.png)

In the last lesson, we gave Claude access to a tool, Claude wanted to call it, and then we actually called the underlying tool function.

In this lesson, we're going to "trick" Claude by telling it about a particular tool, but we won't need to actually call the underlying tool function.  We're using the tool as a way of forcing a particular structure of response, as seen in this diagram:

![structured_response.png](./images/structured_response.png)

## Sentiment analysis
Let's start with a simple example.  Suppose we want Claude to analyze the sentiment in some text and respond with a JSON object that follows this shape: 

```
{
  "negative_score": 0.6,
  "neutral_score": 0.3,
  "positive_score": 0.1
}
```

All we have to do is define a tool that captures this shape using JSON Schema.  Here's a potential implementation: 

In [1]:
%pip install -qU pip
%pip install -qUr requirements.txt

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import boto3
import json
from datetime import datetime
from botocore.exceptions import ClientError

# Import the hints module from the utils package
from utils import hints

session = boto3.Session()
region = session.region_name

In [3]:
modelId = 'anthropic.claude-3-sonnet-20240229-v1:0'
#modelId = 'anthropic.claude-3-haiku-20240307-v1:0'

bedrock_client = boto3.client(service_name = 'bedrock-runtime', region_name = region,)

In [4]:
tools = {
  "tools": [
    {
      "toolSpec": {
        "name": "print_sentiment_scores",
        "description": "Prints the sentiment scores of a given text.",
        "inputSchema": {
          "json": {
            "type": "object",
            "properties": {
              "positive_score": {
                "type": "number",
                "description": "The positive sentiment score, ranging from 0.0 to 1.0."},
              "negative_score": {
                "type": "number",
                "description": "The negative sentiment score, ranging from 0.0 to 1.0."},
              "neutral_score": {
                "type": "number",
                "description": "The neutral sentiment score, ranging from 0.0 to 1.0."}
            },
            "required": ["positive_score", "negative_score", "neutral_score"
            ]
          }
        }
      }
    }
  ]
}

Now we can tell Claude about this tool and explicitly tell Claude to use it, to ensure that it actually does use it.  We should get a response telling us that Claude wants to use a tool.  The tool use response should contain all the data in the exact format we want.

In [5]:
tweet = "I'm a HUGE hater of pickles.  I actually despise pickles.  They are garbage."

query = f"""
<text>
{tweet}
</text>

Only use the print_sentiment_scores tool.
"""

converse_api_params = {
    "modelId": "anthropic.claude-3-haiku-20240307-v1:0",
    "messages": [{"role": "user", "content": [{"text": query}]}],
    "inferenceConfig": {"temperature": 0.0, "maxTokens": 400},
    "toolConfig": tools,
}

response = bedrock_client.converse(**converse_api_params)

In [6]:
response['output']

{'message': {'role': 'assistant',
  'content': [{'text': "Okay, let's analyze the sentiment of the given text using the print_sentiment_scores tool:"},
   {'toolUse': {'toolUseId': 'tooluse_-I83KQP2QSymV-_Kj-_a6w',
     'name': 'print_sentiment_scores',
     'input': {'text': "I'm a HUGE hater of pickles.  I actually despise pickles.  They are garbage."}}}]}}

Let's take a look at the response we get back from Claude.  We've bolded the important part:

>{'message': {'role': 'assistant',
  'content': [{'text': 'Here is the sentiment analysis for the given text:'},
   {'toolUse': {'toolUseId': 'tooluse_d2ReNcjDQvKjLLet4u9EOA',
     'name': 'print_sentiment_scores',
     **'input': {'positive_score': 0.0,
      'negative_score': 0.7,
      'neutral_score': 0.3}**}}]}}

Claude "thinks" it's calling a tool that will use this sentiment analysis data, but really we're just going to extract the data and turn it into JSON:

In [7]:
import json
json_sentiment = None
for content in response['output']['message']['content']:
    if isinstance(content, dict) and 'toolUse' in content:
        tool_use = content['toolUse']
        if tool_use['name'] == "print_sentiment_scores":
            json_sentiment = tool_use['input']
            break

if json_sentiment:
    print("Sentiment Analysis (JSON):")
    print(json.dumps(json_sentiment, indent=2))
else:
    print("No sentiment analysis found in the response.")

Sentiment Analysis (JSON):
{
  "text": "I'm a HUGE hater of pickles.  I actually despise pickles.  They are garbage."
}


It works! Now let's turn that into a reusable function that takes a tweet or article and then prints or returns the sentiment analysis as JSON.

In [8]:
def analyze_sentiment(content):

    query = f"""
    <text>
    {content}
    </text>

    Only use the print_sentiment_scores tool.
    """

    converse_api_params = {
        "modelId": "anthropic.claude-3-haiku-20240307-v1:0",
        "messages": [{"role": "user", "content": [{"text": query}]}],
        "inferenceConfig": {"temperature": 0.0, "maxTokens": 4096},
        "toolConfig": tools,
    }

    response = bedrock_client.converse(**converse_api_params)

    json_sentiment = None
    for content in response['output']['message']['content']:
        if isinstance(content, dict) and 'toolUse' in content:
            tool_use = content['toolUse']
            if tool_use['name'] == "print_sentiment_scores":
                json_sentiment = tool_use['input']
                break

    if json_sentiment:
        print("Sentiment Analysis (JSON):")
        print(json.dumps(json_sentiment, indent=2))
    else:
        print("No sentiment analysis found in the response.")

In [9]:
analyze_sentiment("OMG I absolutely love taking bubble baths soooo much!!!!")

Sentiment Analysis (JSON):
{
  "positive_score": 0.9,
  "negative_score": 0.0,
  "neutral_score": 0.1
}


In [10]:
analyze_sentiment("Honestly I have no opinion on taking baths")

Sentiment Analysis (JSON):
{
  "positive_score": 0.0,
  "negative_score": 0.0,
  "neutral_score": 1.0
}


***

## Forcing tool use with `toolChoice` 

Currently we're "forcing" Claude to use our `print_sentiment_scores` tool through prompting.  In our prompt, we write `Only use the print_sentiment_scores tool.` which usually works, but there's a better way! We can actually force Claude to use a specific tool using the `tool_choice` parameter:

```json
tool_choice = {
    "tool": {
        "name": "print_sentiment_scores"}
}
```

The above code tells Claude that it must respond by calling the `print_sentiment_scores` tool.  Let's update our tools and function to use it:

In [11]:
# create out toolConfig var and force the toolChoice to be print_sentiment_scores by name
toolConfig = {'tools': [],
        "toolChoice": {
        "tool": {"name":"print_sentiment_scores"},
    }
}

In [12]:
# append our tool specification to our toolConfig
toolConfig['tools'].append({
      "toolSpec": {
        "name": "print_sentiment_scores",
        "description": "Prints the sentiment scores of a given text.",
        "inputSchema": {
          "json": {
            "type": "object",
            "properties": {
              "positive_score": {
                "type": "number",
                "description": "The positive sentiment score, ranging from 0.0 to 1.0."},
              "negative_score": {
                "type": "number",
                "description": "The negative sentiment score, ranging from 0.0 to 1.0."},
              "neutral_score": {
                "type": "number",
                "description": "The neutral sentiment score, ranging from 0.0 to 1.0."}
            },
            "required": ["positive_score", "negative_score", "neutral_score"]
          }
        }
      }
    })

In [13]:
# optional uncomment if you want to see the complete toolConfig
toolConfig

{'tools': [{'toolSpec': {'name': 'print_sentiment_scores',
    'description': 'Prints the sentiment scores of a given text.',
    'inputSchema': {'json': {'type': 'object',
      'properties': {'positive_score': {'type': 'number',
        'description': 'The positive sentiment score, ranging from 0.0 to 1.0.'},
       'negative_score': {'type': 'number',
        'description': 'The negative sentiment score, ranging from 0.0 to 1.0.'},
       'neutral_score': {'type': 'number',
        'description': 'The neutral sentiment score, ranging from 0.0 to 1.0.'}},
      'required': ['positive_score', 'negative_score', 'neutral_score']}}}}],
 'toolChoice': {'tool': {'name': 'print_sentiment_scores'}}}

In [14]:
def analyze_sentiment(content):

    query = f"""
    <text>
    {content}
    </text>

    Only use the print_sentiment_scores tool.
    """

    converse_api_params = {
        "modelId": "anthropic.claude-3-haiku-20240307-v1:0",
        "messages": [{"role": "user", "content": [{"text": query}]}],
        "inferenceConfig": {"temperature": 0.0, "maxTokens": 4096},
        "toolConfig": toolConfig
    }

    response = bedrock_client.converse(**converse_api_params)

    json_sentiment = None
    for content in response['output']['message']['content']:
        if isinstance(content, dict) and 'toolUse' in content:
            tool_use = content['toolUse']
            if tool_use['name'] == "print_sentiment_scores":
                json_sentiment = tool_use['input']
                break

    if json_sentiment:
        print("Sentiment Analysis (JSON):")
        print(json.dumps(json_sentiment, indent=2))
    else:
        print("No sentiment analysis found in the response.")

In [15]:
analyze_sentiment("Honestly I have no opinion on taking baths")

Sentiment Analysis (JSON):
{
  "positive_score": 0.0,
  "negative_score": 0.0,
  "neutral_score": 1.0
}


We'll cover `toolChoice` in greater detail in an upcoming lesson.

***

## Entity extraction example

Let's use this same approach to get Claude to generate nicely formatted JSON that contains entities like people, organizations, and locations extracted from a text sample:


In [16]:
toolConfig = {
  "tools": [
    {
      "toolSpec": {
        "name": "print_entities",
        "description": "Prints extract named entities.",
        "inputSchema": {
          "json": {
            "type": "object",
            "properties": {
              "entities": {
                "type": "array",
                "items": {
                  "type": "object",
                  "properties": {
                    "name": {"type": "string", "description": "The extracted entity name."},
                    "type": {"type": "string", "description": "The entity type (e.g., PERSON, ORGANIZATION, LOCATION)."},
                    "context": {"type": "string", "description": "The context in which the entity appears in the text."}
                  },
                  "required": ["name", "type", "context"]
                }
              }
            },
            "required": ["entities"]
          }
        }
      }
    }
  ]
}

text = "John works at Google in New York. He met with Sarah, the CEO of Acme Inc., last week in San Francisco."

query = f"""
<document>
{text}
</document>

Use the print_entities tool.
"""

converse_api_params = {
    "modelId": "anthropic.claude-3-haiku-20240307-v1:0",
    "messages": [{"role": "user", "content": [{"text": query}]}],
    "additionalModelRequestFields": {"max_tokens": 4096},
    "toolConfig": toolConfig
}

response = bedrock_client.converse(**converse_api_params)


json_entities = None
for content in response['output']['message']['content']:
    if isinstance(content, dict) and 'toolUse' in content:
        tool_use = content['toolUse']
        if tool_use['name'] == "print_entities":
            json_entities = tool_use['input']
            break

if json_entities:
    print("Extracted Entities (JSON):")
    print(json.dumps(json_entities, indent=2))
else:
    print("No entities found in the response.")

Extracted Entities (JSON):
{
  "entities": [
    {
      "name": "John",
      "type": "PERSON",
      "context": "John works at Google in New York."
    },
    {
      "name": "Google",
      "type": "ORGANIZATION",
      "context": "John works at Google in New York."
    },
    {
      "name": "New York",
      "type": "LOCATION",
      "context": "John works at Google in New York."
    },
    {
      "name": "Sarah",
      "type": "PERSON",
      "context": "He met with Sarah, the CEO of Acme Inc., last week in San Francisco."
    },
    {
      "name": "Acme Inc.",
      "type": "ORGANIZATION",
      "context": "He met with Sarah, the CEO of Acme Inc., last week in San Francisco."
    },
    {
      "name": "San Francisco",
      "type": "LOCATION",
      "context": "He met with Sarah, the CEO of Acme Inc., last week in San Francisco."
    }
  ]
}


We're using the same "trick" as before.  We tell Claude it has access to a tool as a way of getting Claude to respond with a particular data format.  Then we extract the formatted data Claude responded with, and we're good to go. 

Remember that in this use case, it helps to explicitly tell Claude we want it to use a given tool:


>Use the print_entities tool.


***

## Wikipedia summary example with more complex data

Let's try another example that's a little more complex.  We'll use the Python `wikipedia` package to get entire Wikipedia page articles and pass them to Claude.  We'll use Claude to generate a response that includes:

* The main subject of the article
* A summary of the article
* A list of keywords and topics mentioned in the article
* A list of category classifications for the article (entertainment, politics, business, etc.) along with a classification score (i.e., how strongly the topic falls into that category)

If we passed Claude the Wikipedia article about Walt Disney, we might expect a result like this: 

```json
{
  "subject": "Walt Disney",
  "summary": "Walter Elias Disney was an American animator, film producer, and entrepreneur. He was a pioneer of the American animation industry and introduced several developments in the production of cartoons. He held the record for most Academy Awards earned and nominations by an individual. He was also involved in the development of Disneyland and other theme parks, as well as television programs.",
  "keywords": [
    "Walt Disney",
    "animation",
    "film producer",
    "entrepreneur",
    "Disneyland",
    "theme parks",
    "television"
  ],
  "categories": [
    {
      "name": "Entertainment",
      "score": 0.9
    },
    {
      "name": "Business",
      "score": 0.7
    },
    {
      "name": "Technology",
      "score": 0.6
    }
  ]
}
```

Here's an example implementation of a function that expects a Wikipedia page subject, finds the article, downloads the contents, passes it to Claude, and then prints out the resulting JSON data.  We use the same strategy of defining a tool to "coach" the shape of Claude's response.

Note: make sure to `pip install wikipedia` if you don't have it on your machine!

In [17]:
import wikipedia

#tool definition
toolConfig = {
  "tools": [
    {
      "toolSpec": {
        "name": "print_article_classification",
        "description": "Prints the classification results.",
        "inputSchema": {
          "json": {
            "type": "object",
            "properties": {
              "subject": {
                "type": "string",
                "description": "The overall subject of the article"},
              "summary": {
                "type": "string",
                "description": "A paragaph summary of the article"},
              "keywords": {
                "type": "array",
                "items": {
                  "type": "string",
                  "description": "List of keywords and topics in the article"}
              },
              "categories": {
                "type": "array",
                "items": {
                  "type": "object",
                  "properties": {
                    "name": {"type": "string", "description": "The category name."},
                    "score": {"type": "number", "description": "The classification score for the category, ranging from 0.0 to 1.0."}
                  },
                  "required": ["name", "score"]
                }
              }
            },
            "required": ["subject", "summary", "keywords", "categories"]
          }
        }
      }
    }
  ]
}

#The function that generates the json for a given article subject
def generate_json_for_article(subject):
    page = wikipedia.page(subject, auto_suggest=True)
    query = f"""
    <document>
    {page.content}
    </document>

    Use the print_article_classification tool. Example categories are Politics, Sports, Technology, Entertainment, Business.
    """

    converse_api_params = {
        "modelId": "anthropic.claude-3-haiku-20240307-v1:0",
        "messages": [{"role": "user", "content": [{"text": query}]}],
        "additionalModelRequestFields": {"max_tokens": 4096},
        "toolConfig": toolConfig
    }

    response = bedrock_client.converse(**converse_api_params)

    json_classification = None
    for content in response['output']['message']['content']:
        if isinstance(content, dict) and 'toolUse' in content:
            tool_use = content['toolUse']
            if tool_use['name'] == "print_article_classification":
                json_classification = tool_use['input']
                break

    if json_classification:
        print("Text Classification (JSON):")
        print(json.dumps(json_classification, indent=2))
    else:
        print("No text classification found in the response.")

In [18]:
generate_json_for_article("Jeff Goldblum")

Text Classification (JSON):
{
  "subject": "Jeff Goldblum",
  "summary": "The article provides a detailed biography of American actor and musician Jeff Goldblum. It covers his early life, acting career, filmography, personal life, and musical pursuits. Goldblum is known for starring in major blockbuster films like Jurassic Park and Independence Day, as well as his work in various Wes Anderson movies. The article highlights his versatility as an actor, his jazz music career, and his personal life including his marriages and children.",
  "keywords": [
    "Jeff Goldblum",
    "actor",
    "musician",
    "Jurassic Park",
    "Independence Day",
    "Wes Anderson",
    "jazz",
    "personal life"
  ],
  "categories": [
    {
      "name": "Entertainment",
      "score": 0.9
    },
    {
      "name": "Arts & Culture",
      "score": 0.7
    }
  ]
}


In [19]:
generate_json_for_article("Octopus")

Text Classification (JSON):
{
  "subject": "Octopus",
  "summary": "The article provides a comprehensive overview of octopuses, including their anatomy, physiology, life cycle, distribution, behavior, ecology, evolution, and relationship to humans. It covers topics such as their intelligence, camouflage, locomotion, feeding, defense mechanisms, and cultural significance in art, literature, and mythology.",
  "keywords": [
    "octopus",
    "cephalopod",
    "invertebrate",
    "marine",
    "anatomy",
    "physiology",
    "life cycle",
    "distribution",
    "habitat",
    "behavior",
    "ecology",
    "evolution",
    "intelligence",
    "camouflage",
    "locomotion",
    "feeding",
    "defense",
    "art",
    "literature",
    "mythology",
    "food source"
  ],
  "categories": [
    {
      "name": "Biology",
      "score": 0.9
    },
    {
      "name": "Zoology",
      "score": 0.8
    },
    {
      "name": "Marine Biology",
      "score": 0.8
    },
    {
      "name": "N

In [20]:
generate_json_for_article("Herbert Hoover")

Text Classification (JSON):
{
  "subject": "Herbert Hoover",
  "summary": "This document provides a detailed biography of Herbert Hoover, the 31st president of the United States. It covers his early life, career as a mining engineer and humanitarian, tenure as Secretary of Commerce, presidency during the Great Depression, post-presidency activities, and legacy.",
  "keywords": [
    "Herbert Hoover",
    "31st president",
    "Great Depression",
    "mining engineer",
    "humanitarian",
    "Secretary of Commerce",
    "Reconstruction Finance Corporation",
    "Smoot-Hawley Tariff Act"
  ],
  "categories": [
    {
      "name": "Politics",
      "score": 0.9
    },
    {
      "name": "History",
      "score": 0.8
    },
    {
      "name": "Business",
      "score": 0.7
    }
  ]
}


***

## Exercise

Use the above strategy to write a function called `translate` that takes a word or phrase and generates a structured JSON output that includes the original phrase in English and the translated phrase in Spanish, French, Japanese, and Arabic.

Here is an example of how this should work:

If we call this:

In [26]:
translate("how much does this cost")

**Step 1.** You'll need to complete a toolConfig with a toolSpec for a tool we'll name "translations_from_claude". Here is the start of the toolSpec which includes English

In [24]:
# Here is a strater toolConfig we've added "English" for you.
toolConfig = {
  "tools": [
    {
      "toolSpec": {
        "name": "translations_from_claude",
        "description": "The translations from Claude of a user provided phrase into English to Spanish, French, Japanese, and Arabic.",
        "inputSchema": {
          "json": {
            "type": "object",
            "properties": {
              "english": {"type": "string", "description": "Your English translation of the provided content from the user"},
            },
            "required": ["english"]
          }
        }
      }
    }
  ],
    "toolChoice": {"tool": {"name": "translations_from_claude"}}
}

❓ If you want a hint related to your toolSpec, run the cell below!

In [23]:
print(hints.exercise_10_2_2_toolSpec)


toolConfig = {
  "tools": [
    {
      "toolSpec": {
        "name": "translations_from_claude",
        "description": "The translations from Claude of a user provided phrase into English to Spanish, French, Japanese, and Arabic.",
        "inputSchema": {
          "json": {
            "type": "object",
            "properties": {
              "english": {"type": "string", "description": "Your English translation of the provided content from the user"},
              "spanish": {"type": "string", "description": "Your Spanish translation of the provided content from the user"},
              "french": {"type": "string", "description": "Your French translation of the provided content from the user"},
              "japanese": {"type": "string", "description": "Your Japanese translation of the provided content from the user"},
              "arabic": {"type": "string", "description": "Your Arabic translation of the provided content from the user"}
            },
            "require

**Step 2.** Now you need to complete your "translate" function.

In [27]:
def translate(query):
    prompt = f"""
    Translate phrase below from the user into Spanish, French, Japanese and Arabic.
    Content to translate: '{query}'
    """

    pass

❓ If you want a hint related to your translate function, run the cell below!

In [28]:
print(hints.exercise_10_2_2_translate)


def translate(query):
    prompt = f"Translate this phrase from the user into English, Spanish, French, Japanese and Arabic. Content to translate: '{query}'"

    converse_api_params = {
        "modelId": modelId,
        "messages": [{"role": "user", "content": [{"text": prompt}]}],
        "additionalModelRequestFields": {"max_tokens": 4096},
        "toolConfig": toolConfig
    }

    response = bedrock_client.converse(**converse_api_params)
    tool_use = response['output']['message']['content'][-1]
    translations_from_claude = tool_use['toolUse']['input']

    print(json.dumps(translations_from_claude, ensure_ascii=False, indent=2))



**Step 3.** It's time to give our translate function a try.

In [29]:
translate("how much does this cost")

We expect an output like this: 

```json
{
  "english": "how much does this cost",
  "spanish": "¿cuánto cuesta esto?",
  "french": "combien ça coûte?",
  "japanese": "これはいくらですか",
  "arabic": "كم تكلفة هذا؟"
}
```

**NOTE: If you want to print your results, this line of code will help you print them out nicely:**

In [ ]:
print(json.dumps(translations_from_claude, ensure_ascii=False, indent=2))